In [186]:
# Чтобы скачать и распаковать zip-архив в output папку в Питоне:
import pandas as pd
import numpy as np
from IPython.display import display
#!/usr/bin/env python
import io
import zipfile
import requests  # $ pip install requests

r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/8e23b4ad7fb8d49f0b9c67be84dc440c/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/ratings_movies.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('Data')

In [193]:
df = pd.read_csv('Data/ratings_movies.csv', sep=',')
df = df.drop('Unnamed: 0', axis=1)
display(df.head())
df['date'] = pd.to_datetime(df['date'])
# df['userId'] = df['userId'].astype('category')
# df['genres'] = df['genres'].astype('category')
df.info()


,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   userId   100836 non-null  int64         
 1   movieId  100836 non-null  int64         
 2   rating   100836 non-null  float64       
 3   date     100836 non-null  datetime64[ns]
 4   title    100836 non-null  object        
 5   genres   100836 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.6+ MB


In [194]:
# Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).
# библиотека для регулярных выражений

import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return 0
 
# Задание 8.1
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска? 
    
df['year_release'] = df['title'].apply(get_year_release)
mask_0 = df['year_release'] == 0 
display(df[mask_0]['movieId'].count())
df.info()


18

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   userId        100836 non-null  int64         
 1   movieId       100836 non-null  int64         
 2   rating        100836 non-null  float64       
 3   date          100836 non-null  datetime64[ns]
 4   title         100836 non-null  object        
 5   genres        100836 non-null  object        
 6   year_release  100836 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [196]:
# Задание 8.2
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
# В качестве ответа запишите название этого фильма без указания года его выпуска.

mask_year_1999 = df['year_release'] == 1999
df_1999 =df[mask_year_1999]
type(df_1999.head())
type(df_1999.groupby(by=['movieId'])['rating'].mean())

pandas.core.series.Series